<a href="https://colab.research.google.com/github/JoseLuisPalominoMarmol/ModeloClasificacionTexto.ipynb/blob/main/ModeloClasificacionTexto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creando un modelo de clasificación de texto

## Instalando librerías

En primer lugar, instalamos las siguientes librerías de [HuggingFace](https://huggingface.co/): [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).

In [ ]:
pip install datasets evaluate transformers[sentencepiece] accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.9 MB/s eta 0:00:00


Si estás en Google Colab, después de ejecutar la celda anterior debes reiniciar el entorno desde el menú Runtime -> Restart Session. A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

El primer paso es la elección de un dataset. Para este ejemplo vamos a utilizar el [dataset de senti_lex](https://huggingface.co/datasets/senti_lex) que contiene léxicos de sentimiento generados mediante propagación en un grafo basado de conocimiento. Clasifica una palabra en positiva o negativa. Para cada una de las palabras se incluye una valoración entre 0 (negativa) y 1 (positiva). Nuestro objetivo es crear un modelo para clasificar de manera automática palabras en función de si son consideradas negativas o positvas.

Comenzamos descargando el dataset.

In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset("senti_lex", "es")

Veamos el contenido de este dataset.

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['word', 'sentiment'],
        num_rows: 4275
    })
})

En ocasiones, el dataset puede estar previamente segmentado en grupos de entrenamiento y prueba; sin embargo, en esta instancia, no es así, lo que implica que necesitaremos hacer la división por nosotros mismos. El atributo `train` incluye dos columnas: `word` y `sentiment`. Si deseamos examinar los datos del conjunto de entrenamiento, podemos convertirlos a formato de pandas para visualizarlos en forma de tabla.

In [ ]:
raw_dataset['train'].to_pandas()

,word,sentiment
0,en,0
1,para,0
2,sin,0
3,tiempo,0
4,bajo,0
...,...,...
4270,prudentemente,1
4271,ondear,1
4272,chic,1
4273,subvencionar,1


Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment).

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

model_checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Definimos una función para tokenizar el texto. En este caso, tokenizaremos la columna `word`.

In [ ]:
def tokenize_function(example):
    return tokenizer(example["word"], truncation=True)

Tokenizamos el dataset y lo mostramos.

In [ ]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/4275 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4275
    })
})

Podemos ver que han aparecido tres nuevas columnas ('input_ids', 'token_type_ids' y 'attention_mask') que serán utilizadas para entrenar el modelo.

Para poder entrenar un modelo de clasificación de texto, es necesario que nuestro dataset tenga una columna llamada `label`, por lo que tenemos que renombrar nuestra columna `sentiment`.

In [ ]:
tokenized_dataset = tokenized_dataset.rename_column('sentiment','label')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['word', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4275
    })
})

Además, necesitamos partir nuestro dataset en un conjunto de entrenamiento y en un conjunto de test. Para lo cual, vamos a:

- Revolver el dataset.
- Calcular el número de elementos de nuestro dataset.
- Dividir el dataset en dos trozos (80% para entrenar y 20% para testear).
- Construir un nuevo dataset con un conjunto de entrenamiento y uno de test.

In [ ]:
from datasets import DatasetDict,Dataset
# 1. Revolvemos el dataset con el método shuffle
new_tokenized_dataset = tokenized_dataset["train"].shuffle()
# 2. Calculamos el número de elementos del dataset
len_dataset = len(tokenized_dataset["train"])
# 3. Partimos el dataset en dos trozos
train_dataset = tokenized_dataset["train"][0:int(len_dataset*0.8)]
test_dataset = tokenized_dataset["train"][int(len_dataset*0.8):]
# 4.  Creamos un Nuevo DatasetDict con los trozos divididos
new_dataset = DatasetDict({"train":Dataset.from_dict(train_dataset),"test":Dataset.from_dict(test_dataset)})

Por último, antes de definir nuestro modelo tenemos que definir una función que se va a encargar de preparar los datos para que sean procesados de manera eficiente por el modelo.

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Modelo

Pasamos ahora a definir el modelo, lo primero que vamos a definir son los argumentos con los que vamos a entrenar nuestro modelo. Aunque podemos configurar el entrenamiento de múltiples maneras, en este caso vamos a utilizar los valores por defecto, y solo vamos a modificar el nombre con el que se va a guardar nuestro modelo, que en este caso va a ser `nlptown/bert-base-multilingual-uncased-sentiment`. Además le vamos a pedir que nos muestre cómo de bien funciona el modelo a medida que se va entrenando mediante la `evaluation_strategy` con valor `epoch`.

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("nlptown/bert-base-multilingual-uncased-sentiment",evaluation_strategy="epoch")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


A continuación definimos nuestro modelo, para ello usamos la clase `AutoModelForSequenceClassification` y vamos a utilizar un modelo pre-entrenado (recordar lo que era el transfer learning). Para ello solo tenemos que indicar el nombre de nuestro modelo (definido previamente en la variable `model_checkpoint` y el número de posibles valores que puede tomar nuestro clasificador (en este caso 2, negativo y positivo).  

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)

Ahora definimos la función que usaremos para calcular la precisión de nuestro modelo. En este caso usaremos la accuracy.

In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

Ya podemos definir nuestro objeto `trainer` que usaremos para entrenar nuestro modelo. La estructura de este objeto será siempre la misma. Le tenemos que proporcionar:
1. El modelo.
2. La configuración del entrenamiento.
3. El conjunto de entrenamiento.
4. El conjunto de test.
5. El objeto que prepara los datos.
6. El tokenizador.
7. La métrica.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=new_dataset["train"],
    eval_dataset=new_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Y ahora entrenamos el modelo mediante el método `train`. Este proceso puede llevar unos minutos y entrenará el modelo por 3 épocas (es decir mostrará todos los datos al modelo 3 veces). Este valor se puede cambiar en [la configuración del entrenamiento](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments).  

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.852435,0.307602
2,0.394100,2.372604,0.560234
3,0.196200,3.237254,0.444444


TrainOutput(global_step=1284, training_loss=0.25301999986357404, metrics={'train_runtime': 204.0966, 'train_samples_per_second': 50.27, 'train_steps_per_second': 6.291, 'total_flos': 28842270757128.0, 'train_loss': 0.25301999986357404, 'epoch': 3.0})

## Compartiendo el modelo

Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [ ]:
# Vamos a la carpeta donde se ha guardado nuestro modelo, es el valor que
# definimos previamente en el objeto TrainingArguments
%cd nlptown/bert-base-multilingual-uncased-sentiment
# Subimos el modelo indicando un mensaje de confirmación, y una etiqueta.
trainer.push_to_hub(commit_message="Training complete", tags="classification")

/content/nlptown/bert-base-multilingual-uncased-sentiment


CommitInfo(commit_url='https://huggingface.co/JoseLuis95/bert-base-multilingual-uncased-sentiment/commit/48a9edb46afc563823912a489158ef84f122abea', commit_message='Training complete', commit_description='', oid='48a9edb46afc563823912a489158ef84f122abea', pr_url=None, pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/JoseLuis95/bert-base-multilingual-uncased-sentiment.

Finalmente, vamos a ver cómo usar nuestro modelo para hacer predicciones desde código (esto puede ser útil sí por ejemplo nos interesa procesar múltiples textos de manera secuencial).


## Usando el modelo

En este caso al ser un modelo que hemos entrenado nosotros mismos podríamos usar los ficheros locales, pero vamos a ver cómo usar el modelo que acabamos de subir al hub de HuggingFace.

Para ello usamos un `pipeline` al que le debemos indicar el nombre del modelo que queremos descargar.

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model='JoseLuis95/bert-base-multilingual-uncased-sentiment')

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Ahora podemos hacer predicciones con nuestro modelo que tomará valores de label: 1 star (interpretación de palabra negativa) a label: 2 stars (interpretación de palabra positiva).

In [ ]:
classifier('Brillante')

[{'label': '2 stars', 'score': 0.9975670576095581}]

In [ ]:
classifier('Malicioso')

[{'label': '1 star', 'score': 0.998887836933136}]

In [ ]:
classifier('Desnutrido')

[{'label': '1 star', 'score': 0.9990437626838684}]

In [ ]:
classifier('Inteligente')

[{'label': '2 stars', 'score': 0.9971132278442383}]

In [ ]:
classifier('Vergonzoso')

[{'label': '1 star', 'score': 0.9990097284317017}]